In [ ]:
# google drive
from google.colab import drive
drive.mount('/content/drive')

# base
from pathlib import Path
import pandas as pd
import numpy as np
import random
import os

# CV
from sklearn.model_selection import KFold, StratifiedKFold

# Imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# CLFs
import lightgbm as lgb
from sklearn.linear_model import BayesianRidge, Ridge, LinearRegression
from sklearn.kernel_approximation import Nystroem
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

# metrics
from sklearn.metrics import mean_squared_error

from tqdm import tqdm

import yaml
from drive.MyDrive.Kaggle.Imputation_062022.src.functions import *

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Kaggle/Imputation_062022/src/data/data.csv', index_col='row_id')
sample = pd.read_csv('/content/drive/MyDrive/Kaggle/Imputation_062022/src/sample_submission.csv', index_col='row-col')

In [ ]:
set_seed(seed=69)
col_list, F1, F2, F3, F4, missing_cols = get_lists(data)

In [ ]:
F4data = data[F4]
F4data['n_missing'] = F4data.isnull().sum(axis=1)
F4data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14,n_missing
row_id,,,,,,,,,,,,,,,,
0,5.547214,1.066871,-0.134313,-0.101040,-0.660871,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055,0
1,-1.707374,-1.188114,-0.562419,-1.462988,1.290672,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775,0
2,1.914908,3.877128,NaN,0.358635,0.443973,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576,1
3,-2.638262,0.546676,0.865400,-0.857077,2.667105,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152,0
4,-0.230342,-0.459019,1.128705,-0.748683,NaN,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1.148096,-5.126425,0.746223,-0.195402,2.615170,1.799592,-0.301352,5.339675,-0.991529,1.279494,-0.841051,-2.276500,1.762961,5.324553,-0.228733,0
999996,-4.990146,-1.636969,0.862797,0.331960,2.386669,1.909697,-1.299360,-0.071713,-0.162173,0.072501,-0.614687,-1.265524,0.190385,-0.344112,-0.346807,0
999997,1.067439,0.293753,-0.866815,-0.291116,1.208282,2.891854,3.105002,-3.470520,NaN,0.096988,0.569255,3.609790,-0.584108,-1.492096,-0.997502,1


In [ ]:
%%time
base_model = HistGradientBoostingRegressor()


ii = IterativeImputer(estimator=base_model,
                      max_iter=100,
                      verbose=1,
                      initial_strategy='mean'
                      )
ii_df = pd.DataFrame(ii.fit_transform(F4data[F4]), columns=F4data[F4].columns)

[IterativeImputer] Completing matrix with shape (1000000, 15)
[IterativeImputer] Change: 19.61171147758984, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 8.010776755103915, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 4.697555499887629, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 4.2231678299570685, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 2.497192737577551, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 3.2762718261510813, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 3.6159786599030825, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 2.6802906976888456, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 3.913979039680082, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 3.03657796266823, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 2.4104006230108888, scaled tolerance: 0.031229363 
[IterativeImputer] Change: 2.6712892339582712, scaled tolerance: 0.031229363 
[Iterativ

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


CPU times: user 9h 7min 40s, sys: 3min 6s, total: 9h 10min 46s
Wall time: 5h 7min 38s


In [ ]:
for col in F1+F3:
    data[col] = data[col].fillna(np.mean(data[col]))
data[F4] = ii_df

In [ ]:
sub = make_submission(data, sample)
sub.to_csv('drive/MyDrive/Kaggle/Imputation_062022/src/II_HGBR.csv', index='row-col')

In [ ]:
ii_df

,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
0,5.547214,1.066871,-0.134313,-0.101040,-0.660871,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,-1.707374,-1.188114,-0.562419,-1.462988,1.290672,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,1.914908,3.877128,0.359208,0.358635,0.443973,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-2.638262,0.546676,0.865400,-0.857077,2.667105,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,-0.230342,-0.459019,1.128705,-0.748683,-0.302047,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1.148096,-5.126425,0.746223,-0.195402,2.615170,1.799592,-0.301352,5.339675,-0.991529,1.279494,-0.841051,-2.276500,1.762961,5.324553,-0.228733
999996,-4.990146,-1.636969,0.862797,0.331960,2.386669,1.909697,-1.299360,-0.071713,-0.162173,0.072501,-0.614687,-1.265524,0.190385,-0.344112,-0.346807
999997,1.067439,0.293753,-0.866815,-0.291116,1.208282,2.891854,3.105002,-3.470520,-0.183995,0.096988,0.569255,3.609790,-0.584108,-1.492096,-0.997502
999998,-0.863684,3.805997,-0.189223,-0.864603,-2.608098,-1.135003,-5.127360,-3.903728,-1.597023,0.893159,0.379434,0.846266,-1.085554,3.122423,0.004831
